In [1]:
#importamos los paquetes necesarios
import torch #contiene todas las funciones de PyTorch
import torch.nn as nn #contiene la clase padre de todos los modelos (nn.Module)
import torch.nn.functional as F #esencial para la función de activación 
import torchvision #fundamental para la importación de imágenes
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
#importamos también matplotlib.pyplot y numpy para la visualización de las imágenes
import matplotlib.pyplot as plt
import numpy as np
import os

#importamos los paquetes necesarios para el cálculo de las métricas
import sklearn
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
import seaborn as sns

#e importamos el paquete para el cálculo del tiempo de ejecución
import time

C:\Users\samue\anaconda3\envs\venv_py39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#establecemos el tamaño del batch, la escala de las imágenes y el número de épocas de entrenamiento
batch = 4
escala = 640
epocas = 50

In [3]:
#a continuación definimos la operación que permitirá transformar las imágenes del repositorio en Tensores que puedan ser empleados por PyTorch
transform = transforms.Compose(
    [transforms.ToTensor(), #transforma la imagen de formato PIL a formato tensor
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), #normaliza el tensor para que la media de sus valores sea 0 y su desviación estándar 0.5
     transforms.Resize((escala, escala))]) #redimensionamos las imágenes

In [4]:
#a continuación cargamos el conjunto de imágenes de train (OCT) y los dos de test (iPhone y Samsung)
OCT = ImageFolder(root = 'Datos/Classified Data/Images/OCT', transform = transform)
print(f'Tamaño del conjunto de datos de train: {len(OCT)}')

Samsung = ImageFolder(root = 'Datos/Classified Data/Images/Samsung', transform = transform)
print(f'Tamaño del conjunto de datos de test de Samsung: {len(Samsung)}')

iPhone = ImageFolder(root = 'Datos/Classified Data/Images/iPhone', transform = transform)
print(f'Tamaño del conjunto de datos de test de iPhone: {len(iPhone)}')

Tamaño del conjunto de datos de train: 113
Tamaño del conjunto de datos de test de Samsung: 93
Tamaño del conjunto de datos de test de iPhone: 99


In [5]:
#establecemos una lista con el nombre de las etiquetas
classes = OCT.classes

In [6]:
#en esta ocasión, debido a que vamos a implementar EarlyStopping es necesario dividir el conjunto de entrenamiento en train y validation
#Dividimos el conjunto de datos en entrenamiento y validación
train_size = int(0.8 * len(OCT))
val_size = len(OCT) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(OCT, [train_size, val_size])

# Crear cargadores de datos para cada conjunto
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = 4, 
    shuffle = True,
    num_workers = 2 #genera subprocesos para cargar los datos y así liberamos el proceso main
)

val_loader = torch.utils.data.DataLoader(
    dataset = val_dataset,
    batch_size = 4,
    shuffle = True,
    num_workers = 2
)

test_S_loader = DataLoader(
    dataset = Samsung,
    batch_size = 4, #establecemos un tamaño de lote (batch_size) de 10, ya que son pocas imágenes y podemos permitírnoslo
    shuffle = True, #indicamos que mezcle las imágenes
    num_workers = 2 #genera subprocesos para cargar los datos y así liberamos el proceso main
)

test_i_loader = DataLoader(
    dataset = iPhone,
    batch_size = 4, #establecemos un tamaño de lote (batch_size) de 10, ya que son pocas imágenes y podemos permitírnoslo
    shuffle = True, #indicamos que mezcle las imágenes
    num_workers = 2 #genera subprocesos para cargar los datos y así liberamos el proceso main
)

In [7]:
#a continuación debemos definir el modelo
#una vez que hemos comprobado que las funciones de carga funcionan correctamente ya podemos definir el modelo
#los modelos se definen como clases que heredan todos ellos de un mismo padre: nn.Module
#las clases contienen 2 funciones básicas: __init__() y forward()
class CNN(nn.Module):
    
    def __init__(self):
        #esta función sobreescribe la función init() del padre
        super(CNN,self).__init__()
        #definimos todas las capas que van a constituir el modelo
        #una primera capa convolucional
        self.conv1 = nn.Conv2d(
            in_channels = 3, #3 canales de entrada porque las imágenes son a color
            out_channels = 6, #se trata del número de salidas de la capa. Es el número de kernels de la capa convolucional
            kernel_size = 5, #suele tratarse de un número impar
            stride = 1, #cantidad píxeles que se desplaza el filtro sobre la imagen
            padding = 0, #cantidad de relleno que se va a aplicar sobre los bordes de la imagen
        )
        #una segunda capa convolucional
        self.conv2 = nn.Conv2d(
            in_channels = 6, #6 canales de entrada porque es el número de salidas de la capa anterior
            out_channels = 16, #se trata del número de salidas de la capa. Es el número de kernels de la capa convolucional
            kernel_size = 5, #suele tratarse de un número impar
            stride = 1, #cantidad píxeles que se desplaza el filtro sobre la imagen
            padding = 0, #cantidad de relleno que se va a aplicar sobre los bordes de la imagen
        )
        
        #una primera capa fully-connected (red neuronal propiamente dicha)
        self.fc1 = nn.Linear(
            in_features = 16*157*157, #número de parámetros de entrada de la red (los valores se obtienen experimentalmente)
            out_features = 120 #número de neuronas de salida
        )
        
        #una segunda fully-connected
        self.fc2 = nn.Linear(120, 84)
        
        #y una tercera. Nótese que el número de neuronas de salida de la última fully-connected ha de coincidir con el número de clases
        self.fc3 = nn.Linear(84,5)
        
    def forward(self,x):
        #en esta función es donde tiene lugar la computación (y la función invocada por defecto al ejecutar la red)
        #primero aplicamos la función ReLU a la capa convolucional, que simplifica los datos. 
        #ReLU Interpreta los valores positivos como son, y los negativos los torna 0, permitiendo acelerar el entrenamiento
        #al resultado le aplicamos MaxPooling que reduce las dimensiones de los datos, seleccionando el valor máximo del kernel.
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size = 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size = 2)
        #aplanamos la salida, hasta convertirla de forma matricial a forma vectorial (sería la capa flatten)
        x = x.view(-1,self.num_flat_features(x))#usamos una función propia de la clase para obtener el número de características
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) #no incluimos una capa de LogSoft, que convierte el output en probabilidad, ya que la función loss que usaremos incluye esta funcionalidad
        return x
    
    def num_flat_features(self,x):
        #por último definimos la función que permite obtener el número de características de los tensores
        size = x.size()[1:] #seleccionamos todas las dimensiones expcepto la primera (que son los batches)
        num_features = 1
        #va iterando y calcula el número de características de los datos (x)
        for s in size:
            num_features*=s
        return num_features

#una vez definida la clase generamos una instancia de la misma
cnn = CNN()
#y mostramos por pantalla sus secciones
print(cnn)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=394384, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=5, bias=True)
)


In [8]:
#a continuación debemos entrenar el modelo, para ello es necesario definir una función loss que evalúa la desviación entre las predicciones y los valores reales
#definimos como loss la función de tipo cross entropy 
criterion = nn.CrossEntropyLoss() 

#y una función optimizadora que modificará los pesos de la red para tratar de mejorar su rendimiento
#en este caso el optimizador será la función Adam (ampliamente utilizada)
optimizer = torch.optim.Adam(params = cnn.parameters(), #los parámetros son los pesos que deberá ir actualizando el optimizador
                             lr = 0.001) #dejamos el valor de learning rate por defecto (0.001)

HAY UNA MANERA DE HACER EARLYSTOPPING EMPLEANDO LA LIBRERÍA PYTORCHTOOLS, QUE INCLUYE UNA PROPIA FUNCIÓN DE EARLYSTOPPING, PERO PARA SU USO ES NECESARIO QUE LAS DEPENDENCIAS SE ENCUENTREN EN EL MISMO DIRECTORIO QUE EL SCRIPT EN CUESTIÓN, Y ESO EN SCAYLE GENERA PROBLEMAS, es más jaleo. Por eso prefiero hacerlo de una manera más manual

In [ ]:
#vamos a definir la función para el entrenamiento
def entrena_val(red,epocas,paciencia,train_loader,val_loader,optimizer,criterion):
    #primero importamos los paquetes necesarios
    import torch
    #inicializamos best_val_loss (que es el parámetro que va a marcar el Early Stopping) como infinito
    best_val_loss = float('inf')
    #creamos también una variable para almacenar los parámetros del mejor modelo (aquel con menor val_loss)
    best_model_params = None
    #iniciamos también un contador, para poder aplicar Early Stopping con la paciencia deseada
    contador = 0
    #definimos 2 listas en las que almacenaremos los valores de accuracy y loss de train cada época para devolverlas
    acc_graph = []
    loss_graph = []
    #y 2 listas en las que almacenaremos los valores de accuracy y loss de validación cada época para poder devolverlas
    val_acc_graph = []
    val_loss_graph = []
    
    #para entrenar el modelo vamos a iterar el número de épocas determinadas, calculando el valor de loss y accuracy para cada época
    for epoch in range(epocas):
        #establecemos el número de predicciones correctas inicial a 0
        correct = 0
        #y cargamos las imágenes de entrenamiento y sus etiquetas usando la estructura Loader previamente creada
        for data in train_loader:
            inputs, labels = data
            #establecemos a 0 los parámetros del modelo
            optimizer.zero_grad()
            #generamos las predicciones de los inputs
            outputs = red(inputs)
            #calculamos el loss, la desviación de las predicciones con respecto a las etiquetas
            loss = criterion(outputs, labels)
            #propagamos hacia atrás el valor loss
            loss.backward()
            #y modificamos los pesos en función del loss y la función optimizer
            optimizer.step()
            #actualizamos el número de predicciones correctas
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            
        #una vez finalizada la época (que recorre todo el conjunto de imágenes) mostramos el valor del loss y del accuracy
        print(f'Época {epoch +1}/{epocas} - Accuracy: {correct/len(train_loader.dataset)} - Loss: {loss.data.item()}')
        #añadimos los valores a la lista correspondiente
        loss_graph.append(loss.data.item())
        acc_graph.append(correct/len(train_loader.dataset))

        #realizamos ahora las iteraciones correspondientes a las imágenes de validación
        #primero establecemos el valor del loss de validación a cero
        val_loss = 0.0
        #establecemos así mismo el número de predicciones correctas nuevamente a cero
        correct = 0
        #cargamos las imágenes de validación y sus etiquetas
        for data in val_loader:
            inputs, labels = data
            #generamos las predicciones a partir de los inputs
            outputs = red(inputs)
            #calculamos el loss
            loss = criterion(outputs, labels)
            #y lo vamos acumulando
            val_loss += loss.item()
            #finalmente calculamos el número de predicciones correctas
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
        
        #una vez finalizada la época (que recorre todo el conjunto de imágenes) mostramos el valor del loss y del accuracy de validación
        print(f'Época {epoch +1}/{epocas} - Val_accuracy: {correct/len(val_loader.dataset)} - Val_loss: {loss.data.item()}')
        #añadimos los valores a la lista correspondiente
        val_loss_graph.append(loss.data.item())
        val_acc_graph.append(correct/len(val_loader.dataset))
        
        #finalmente solo falta realizar la comprobación del Early Stopping
        #si el valor de val_loss de esta época es inferior al mejor conseguido hasta el momento:
        if val_loss < best_val_loss:
            #entonces actualiza el valor del mejor val_loss (ya que lo que queremos es minimizar este valor)
            best_val_loss = val_loss
            #posteriormente guarda el estado del modelo actual
            best_model_params = red.state_dict()
            #y vuelve a establecer el contador de paciencia a 0
            contador = 0
        #si el valor de val_loss no disminuye (no mejora) con respecto al último mejor:
        else:
            #si se ha llegado al límite de la paciencia establecida detiene el entrenamiento para evitar el sobreentrenamiento
            if contador == paciencia:
                break
            #si aún no ha llegado al límite de la paciencia entonces incrementa el contador en uno y sigue entrenando
            else:
                contador += 1
        
        #finalmente, una vez finalizado el entrenamiento se debe cargar el mejor estado del modelo
        red.load_state_dict(best_model_params)
        
        #y devolver las métricas almacenadas de entrenamiento y validación
        return acc_graph, loss_graph, val_acc_graph, val_loss_graph